## Import Libraries

In [1]:
from os import path
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio
from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import phase, amplitude, sigfilt
from brainpipe.visual import *
from brainpipe.statistics import *
from scipy.stats import *

## PAC Decoding - Low//High Encoding
### For ALL time points

In [2]:
from brainpipe.feat.utils._feat import _manageWindow
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.metrics import roc_auc_score
from numpy.random import permutation

# ============================================================================================
nperm = 1000
conds,steps, subjects = ['low','high'],['odor'],['CHAF','FERJ']
color_codes = ['darkorange','blue']
# ============================================================================================
st = study('Olfacto')
path_phase = path.join(st.path, 'feature/2_PAC_Encoding_EpiPerf_LowHigh/')
save_path = path.join(st.path, 'classified/2_Classif_PAC_Encoding_EpiPerf_LowHigh_1000perm/')
# ============================================================================================
          
for su, step in product(subjects,steps):
    pac_list = []
    #=========================== Load phase files (nfreq, nelec, nwin, ntrial) =================================    
    mat1 = np.load(path.join(path_phase, su+'_'+step+'_'+conds[0]+'_bipo_sel_physFT_pac.npz'))
    names, channels, pha_names = mat1['Mai_RL'], mat1['channels'],mat1['fname']
    ampl_names = ['gamma']
    nampl = 1
    sWidth, sStep = 358, 51
    #PAC arrays (nfreqs x nelecs x npts x ntrials)
    pac_list.append(mat1['pac'][:,:,22:47,:])
    npha,nelecs,npts,ntrials = mat1['pac'][:,:,22:47,:].shape
    _, time = _manageWindow(3584, width=sWidth, step=sStep)
    time = (np.array(time[22:47])/512)-3
    mat2name = path.join(path_phase, su+'_'+step+'_'+conds[1]+'_bipo_sel_physFT_pac.npz')
    if path.isfile(mat2name) == True:
        pac_list.append(np.load(mat2name)['pac'][:,:,22:47,:])
        print (su, 'pac shape: ', [pac.shape for pac in pac_list])

        # =========================== Select phase for 1 elec 1 freq =================================                 
        for elec_num in range(nelecs):
            for ampl,pha in product(range(nampl),range(npha)):
                elec, elec_label = channels[elec_num], names[elec_num]
                pha_name, ampl_name = pha_names[pha], ampl_names[ampl]
                
                #Filenames to save
                name_auc = (save_path+str(ampl)+'_'+ampl_name+'/'+str(pha)+'_'+pha_name+'/auc/'+su +'_auc_'+conds[0].capitalize()+'_'+conds[1].capitalize()+'_'+str(elec_label)+'_('+str(elec_num)+').npy')
                name_perm = (save_path+str(ampl)+'_'+ampl_name+'/'+str(pha)+'_'+pha_name+'/auc/'+su +'_perm_'+conds[0].capitalize()+'_'+conds[1].capitalize()+'_'+str(elec_label)+'_('+str(elec_num)+').npy')
                
                if path.exists(name_auc):
                    print(su,elec_num,ampl,pha,'already computed')
                else:
                    pac_data_elec = []
                    for i,xpac in enumerate(pac_list):
                        pac_data_elec.append(xpac[pha,elec_num].swapaxes(0,1))
                    nwin = xpac.shape[1] #(ntrials x nwins)
                    print('--» processing',su, 'elec', elec_num,'/',nelecs, 'ampl',ampl_name,'pha',pha_name)
                    print ('elec ', elec, 'elec_label ', elec_label,'data',[xpac.shape for xpac in pac_data_elec])

            # =============================  Classification Computation ============================================================           
                   # create a data matrix, concatenate along the trial dimension
                    x = np.concatenate(pac_data_elec, axis=0)
                    print ('Size of the concatenated data: ','Number time windows : ', x.shape[1])
                    y = np.hstack([np.array([i]*len(xpac)) for i, xpac in enumerate(pac_data_elec)])
                    print ('Size of label for classif: ', len(y))

                    auc = np.array([])
                    for t in range(x.shape[1]):
                        X = x[:,t]
                        X = X.reshape(-1, 1)
                        score_rep = []
                        for i in range(10):
                            k = 5
                            skf = SKFold(n_splits=k, random_state=None, shuffle=True)
                            skf.get_n_splits(X, y)
                            score_cv = []
                            for train_index, test_index in skf.split(X, y):
                                clf = LDA()
                                X_train, X_test = X[train_index], X[test_index]
                                y_train, y_test = y[train_index], y[test_index]
                                clf.fit(X=X_train, y=y_train)
                                y_pred = clf.predict(X_test)
                                score_cv.append(roc_auc_score(y_test,y_pred,average='weighted'))
                            score_rep.append(np.mean(score_cv))
                        score_rep = np.asarray(score_rep).reshape(1,len(score_rep))
                        auc = np.vstack((auc, score_rep)) if np.size(auc) else score_rep
                    auc = np.swapaxes(auc,0,1)

                    perm_scores = np.array([])
                    for t in range(x.shape[1]):
                        X = x[:,t]
                        X = X.reshape(-1, 1)
                        perm_rep = []
                        for perm in range(nperm):
                            y_perm = y[permutation(len(y))]
                            score_cv = []
                            for train_index, test_index in skf.split(X, y_perm):
                                clf = LDA()
                                X_train, X_test = X[train_index], X[test_index]
                                y_train, y_test = y_perm[train_index], y_perm[test_index]
                                clf.fit(X=X_train, y=y_train)
                                y_pred = clf.predict(X_test)
                                score_cv.append(roc_auc_score(y_test,y_pred,average='weighted'))
                            perm_rep.append(np.mean(score_cv))
                        perm_rep = np.asarray(perm_rep).reshape(1,len(perm_rep))
                        perm_scores = np.vstack((perm_scores, perm_rep)) if np.size(perm_scores) else perm_rep
                    perm_scores = np.swapaxes(perm_scores,0,1)
                    th_0_05_perm = perm_pvalue2level(perm_scores, p=0.05, maxst=True)
                    th_0_01_perm = perm_pvalue2level(perm_scores, p=0.01, maxst=True)
                    print('th_perm : ', th_0_05_perm[0], th_0_01_perm[0], 'auc_mean', auc.mean())

        # ============================== PLOT phase ANALYSIS + STATS & DECODING ACCURACY ===================================================                
                    xfmt = ScalarFormatter(useMathText=True)
                    xfmt.set_powerlimits((0,3))
                    fig = plt.figure(1,figsize=(7,7))
                    title = 'PAC for '+su+' '+conds[0]+' vs '+conds[1]+' '+str(elec)+' '+str(elec_label)+' ('+str(elec_num)+')'
                    fig.suptitle(title, fontsize=12)

                    # Plot the phase + STATS
                    plt.subplot(211)      
                    BorderPlot(time, x, y=y, kind='sem', alpha=0.2, color=color_codes,linewidth=2, 
                               ncol=1, xlabel='Time (s)',ylabel = r'PAC', legend=conds)
                    rmaxis(plt.gca(), ['right', 'top'])
                    plt.legend(loc=0, handletextpad=0.1, frameon=False)
                    plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

                    # Plot DA for the phase
                    plt.subplot(212)
                    BorderPlot(time, auc, color='b', kind='sd',xlabel='Time (s)', ylim=[0.4,1.], ylabel='Decoding accuracy (%)',linewidth=2, alpha=0.3)
                    rmaxis(plt.gca(), ['right', 'top'])
                    plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
                    plt.plot(time, th_0_05_perm*np.ones(len(time)), '--', color='r', linewidth=2)
                    plot_name = (save_path+str(ampl)+'_'+ampl_name+'/'+str(pha)+'_'+pha_name+'/fig/'+su +'_Pac_'+conds[0]+'_'+conds[1]+'_'+str(elec)+'_'+str(elec_label)+'_('+str(elec_num)+').png')            

                    #Save plots
                    np.save(name_auc, auc)
                    np.save(name_perm, perm_scores)
                    plt.savefig(plot_name, dpi=300, bbox_inches='tight')
                    plt.clf()
                    plt.close()
                    del X, auc, pac_data_elec
        del pac_list

    else: 
        print(su,'pas de condition ',conds[1].capitalize())

-> Olfacto loaded
CHAF pac shape:  [(2, 58, 25, 6), (2, 58, 25, 15)]
--» processing CHAF elec 0 / 58 ampl gamma pha theta
elec  c'2-c'1 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.666666666667 0.783333333333 auc_mean 0.4892
not legend
--» processing CHAF elec 0 / 58 ampl gamma pha alpha
elec  c'2-c'1 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.65 0.783333333333 auc_mean 0.486933333333
not legend
--» processing CHAF elec 1 / 58 ampl gamma pha theta
elec  c'3-c'2 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.666666666667 0.766666666667 auc_mean 0.4808
not legend
--» processing CHAF elec 1 / 58 ampl gamma pha alpha
elec  c'3-c'2 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:

th_perm :  0.633333333333 0.816666666667 auc_mean 0.487133333333
not legend
--» processing CHAF elec 15 / 58 ampl gamma pha alpha
elec  e'3-e'2 elec_label  ACC data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.666666666667 0.816666666667 auc_mean 0.500066666667
not legend
--» processing CHAF elec 16 / 58 ampl gamma pha theta
elec  e'4-e'3 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.666666666667 0.783333333333 auc_mean 0.495533333333
not legend
--» processing CHAF elec 16 / 58 ampl gamma pha alpha
elec  e'4-e'3 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.666666666667 0.783333333333 auc_mean 0.489866666667
not legend
--» processing CHAF elec 17 / 58 ampl gamma pha theta
elec  e'5-e'4 elec_label  SFG data [(6, 25), (15

th_perm :  0.666666666667 0.816666666667 auc_mean 0.492066666667
not legend
--» processing CHAF elec 31 / 58 ampl gamma pha theta
elec  f'8-f'7 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.666666666667 0.816666666667 auc_mean 0.486533333333
not legend
--» processing CHAF elec 31 / 58 ampl gamma pha alpha
elec  f'8-f'7 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.65 0.783333333333 auc_mean 0.489533333333
not legend
--» processing CHAF elec 32 / 58 ampl gamma pha theta
elec  f'9-f'8 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.633333333333 0.783333333333 auc_mean 0.500066666667
not legend
--» processing CHAF elec 32 / 58 ampl gamma pha alpha
elec  f'9-f'8 elec_label  SFG data [(6, 25), (15, 25)]
Siz

th_perm :  0.666666666667 0.8 auc_mean 0.4952
not legend
--» processing CHAF elec 46 / 58 ampl gamma pha alpha
elec  h'9-h'8 elec_label  MFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.65 0.8 auc_mean 0.493866666667
not legend
--» processing CHAF elec 47 / 58 ampl gamma pha theta
elec  h'10-h'9 elec_label  MFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.65 0.783333333333 auc_mean 0.4878
not legend
--» processing CHAF elec 47 / 58 ampl gamma pha alpha
elec  h'10-h'9 elec_label  MFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.666666666667 0.766666666667 auc_mean 0.488133333333
not legend
--» processing CHAF elec 48 / 58 ampl gamma pha theta
elec  i'8-i'7 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windo

th_perm :  0.666666666667 0.733333333333 auc_mean 0.479266666667
not legend
--» processing FERJ elec 4 / 85 ampl gamma pha theta
elec  a10-a9 elec_label  MTG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.675 0.741666666667 auc_mean 0.468833333333
not legend
--» processing FERJ elec 4 / 85 ampl gamma pha alpha
elec  a10-a9 elec_label  MTG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.666666666667 0.741666666667 auc_mean 0.510966666667
not legend
--» processing FERJ elec 5 / 85 ampl gamma pha theta
elec  b2-b1 elec_label  HC data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.683333333333 0.741666666667 auc_mean 0.466066666667
not legend
--» processing FERJ elec 5 / 85 ampl gamma pha alpha
elec  b2-b1 elec_label  HC data [(17, 25), (14, 25)]
Size of th

th_perm :  0.666666666667 0.75 auc_mean 0.506733333333
not legend
--» processing FERJ elec 20 / 85 ampl gamma pha theta
elec  d2-d1 elec_label  PHG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.675 0.741666666667 auc_mean 0.455766666667
not legend
--» processing FERJ elec 20 / 85 ampl gamma pha alpha
elec  d2-d1 elec_label  PHG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.666666666667 0.741666666667 auc_mean 0.473933333333
not legend
--» processing FERJ elec 21 / 85 ampl gamma pha theta
elec  d3-d2 elec_label  PHG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.683333333333 0.766666666667 auc_mean 0.479666666667
not legend
--» processing FERJ elec 21 / 85 ampl gamma pha alpha
elec  d3-d2 elec_label  PHG data [(17, 25), (14, 25)]
Size of the conc

--» processing FERJ elec 35 / 85 ampl gamma pha alpha
elec  d'7-d'6 elec_label  FuG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.675 0.741666666667 auc_mean 0.536466666667
not legend
--» processing FERJ elec 36 / 85 ampl gamma pha theta
elec  d'8-d'7 elec_label  FuG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.675 0.75 auc_mean 0.4664
not legend
--» processing FERJ elec 36 / 85 ampl gamma pha alpha
elec  d'8-d'7 elec_label  FuG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.675 0.75 auc_mean 0.486433333333
not legend
--» processing FERJ elec 37 / 85 ampl gamma pha theta
elec  d'9-d'8 elec_label  FuG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.666666666667 0.741

th_perm :  0.675 0.741666666667 auc_mean 0.474866666667
not legend
--» processing FERJ elec 51 / 85 ampl gamma pha alpha
elec  j2-j1 elec_label  pPirT data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.675 0.741666666667 auc_mean 0.512533333333
not legend
--» processing FERJ elec 52 / 85 ampl gamma pha theta
elec  j3-j2 elec_label  ITG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.675 0.741666666667 auc_mean 0.4909
not legend
--» processing FERJ elec 52 / 85 ampl gamma pha alpha
elec  j3-j2 elec_label  ITG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.666666666667 0.733333333333 auc_mean 0.517033333333
not legend
--» processing FERJ elec 53 / 85 ampl gamma pha theta
elec  j4-j3 elec_label  MTG data [(17, 25), (14, 25)]
Size of the concatenated data:

th_perm :  0.666666666667 0.75 auc_mean 0.469233333333
not legend
--» processing FERJ elec 67 / 85 ampl gamma pha alpha
elec  l10-l9 elec_label  MTG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.675 0.741666666667 auc_mean 0.4858
not legend
--» processing FERJ elec 68 / 85 ampl gamma pha theta
elec  l11-l10 elec_label  MTG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.675 0.75 auc_mean 0.492433333333
not legend
--» processing FERJ elec 68 / 85 ampl gamma pha alpha
elec  l11-l10 elec_label  MTG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.675 0.741666666667 auc_mean 0.478733333333
not legend
--» processing FERJ elec 69 / 85 ampl gamma pha theta
elec  o2-o1 elec_label  OFC data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time win

--» processing FERJ elec 83 / 85 ampl gamma pha theta
elec  t6-t5 elec_label  STG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.666666666667 0.75 auc_mean 0.448366666667
not legend
--» processing FERJ elec 83 / 85 ampl gamma pha alpha
elec  t6-t5 elec_label  STG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.666666666667 0.741666666667 auc_mean 0.465633333333
not legend
--» processing FERJ elec 84 / 85 ampl gamma pha theta
elec  t7-t6 elec_label  STG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  31
th_perm :  0.666666666667 0.733333333333 auc_mean 0.462966666667
not legend
--» processing FERJ elec 84 / 85 ampl gamma pha alpha
elec  t7-t6 elec_label  STG data [(17, 25), (14, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif